# Source Code

In [540]:
import numpy as np


def powerMethod(A, initial=None, epsilon=None, numberOfIterations=100, minEigen=False):
    """
    A function to calculate largest eigen value and eigen vector of a matrix, and approximate relative error

    Args: 
            A (numpy array): Matrix to get eigen value/vector (size n*n)
            initial (numpy array): Vector used as initial guess of eigen vector (size 1*n)
            epsilon (float): Stopping criteria at which eigen value is considered converged
            numberOfIterations (int): Number of iterations untill stopping
            minEigen (boolean): Calculate smallest eigen value or largest

    Returns:
            eigenValue (numpy float)
            eigenVector (numpy array)
            error (float)
    Raises:
            TypeError
            ValueError

    """

    ##Validate input##

    # negative number of iterations
    if numberOfIterations <= 0:
        raise ValueError("Number of iterations should be positive")

    # invalid type of matrix A
    if not isinstance(A, np.ndarray):
        raise TypeError("A should be of type numpy.ndarray")

    # invalid type of vector initial
    if (initial is not None) and (not isinstance(A, np.ndarray)):
        raise TypeError("initial should be of type numpy.ndarray")

    # check initial is 1d array
    if (initial is not None) and (initial.ndim != 1):
        raise TypeError("initial should be 1D array")

    # number of iterations is not integer
    if not isinstance(numberOfIterations, int):
        raise TypeError("Number of iterations should be integer")

    #matrix is not square
    if A.shape[0] != A.shape[1]:
        raise TypeError("Matrix should be square")

    # check size of initial vector
    if (initial is not None) and (initial.shape[0] != A.shape[1]):
        raise TypeError("Length of initial vector should match size of matrix")

    ##Calculate eigen pair##

    # dimension of matrix
    n = A.shape[0]

    if minEigen:
        try:
            A = np.linalg.inv(A)
        except:
            #Matrix is not invertible
            raise TypeError("Matrix should be invertible")

    if initial is None:
        # initial guess for eigen vector
        x = np.random.rand(n)
    else:
        x = initial.copy()
        x = x.astype(np.float32)

    # Relative approximate error between each iteration
    error = None
    # Eigen value calculated at previous iteration
    prevEigen = None

    # iterate to get eigen value/eigen vector
    # Stops after numberOfIterations or after error is less than epsilon
    # whichever comes first
    for i in range(numberOfIterations):
        x = np.dot(A, x)
        eigenValue = x[np.argmax(np.abs(x))]
        # If there isn't any left eigen values
        if eigenValue == 0:
            break
        x /= eigenValue
        if prevEigen is not None:

            # I believe it should be relative error, not relative approximate error
            # but doctor asked us to do it as relative approximate error, so I did it
            #error = abs(eigenValue-prevEigen)
            error = np.abs((eigenValue-prevEigen)/eigenValue)

            if (epsilon is not None) and (error <= epsilon):
                break
        prevEigen = eigenValue

    if(minEigen and eigenValue != 0):
        eigenValue = 1/eigenValue

    # Eigen Value, Eigen Vector, Relative approximate error
    return eigenValue, x, error

In [541]:
def deflate(A, initial=None, epsilon=None, numberOfIterations=100, minEigen=False):
    """
    A function to calculate second largest eigen value and eigen vector of a matrix, and approximate relative error

    Args: 
            A (numpy array): Matrix to get eigen value/vector (size n*n)
            initial (numpy array): Vector used as initial guess of eigen vector (size 1*n)
            epsilon (float): Stopping criteria at which eigen value is considered converged
            numberOfIterations (int): Number of iterations untill stopping
            minEigen (boolean): Calculate smallest eigen value or largest

    Returns:
            deflatedMat (numpy array)
            eigenValue (numpy float)
            eigenVector (numpy array)
            error (numpy float)
    Raises:
            TypeError
            ValueError

    """
    
    #get largest eigen value for A
    eigenValue, eigenVector, error = powerMethod(A, initial, epsilon, numberOfIterations, minEigen)

    eigenVectorNormalized = eigenVector/np.linalg.norm(eigenVector)

    if minEigen:
        try:
            A = np.linalg.inv(A)
        except:
            #Matrix is not invertible
            raise TypeError("Matrix should be invertible")

    # get deflated matrix (B = A - lambda.x*.x*T), where x* is normalized eigen vector
    deflatedMat = A-eigenValue*np.outer(eigenVectorNormalized, eigenVectorNormalized.T)

    # get largest eigen value for deflated matrix
    eigenValue, eigenVector, error = powerMethod(deflatedMat, initial, epsilon, numberOfIterations)

    if minEigen and eigenValue != 0:
        eigenValue = 1/eigenValue

    return deflatedMat, eigenValue, eigenVector, error

# Test Cases

In [542]:
np.set_printoptions(suppress=True)

mat = np.array([[2, -1, 0, 0],
                [-1, 4, -1, 0],
                [0, -1, 4, -1],
                [0, 0, -1, 2]])

# Example of initial guess
init = np.array([2, -1, 0, 0])
e, x, error = powerMethod(mat, initial=init)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat)
print(deflated)
print(e)
print(x)
print(error)

5.302775637731995
[-0.30277564  1.         -1.          0.30277564]
0.0
[[ 1.7773501  -0.26463733 -0.73536267  0.2226499 ]
 [-0.26463733  1.57126208  1.42873792 -0.73536267]
 [-0.73536267  1.42873792  1.57126208 -0.26463733]
 [ 0.2226499  -0.73536267 -0.26463733  1.7773501 ]]
3.618033988749895
[-0.61803399  1.          1.         -0.61803399]
0.0


In [543]:
mat = np.array([[-7, 2],
                [8, -1]])
print(mat)
e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat)
print(deflated)
print(e)
print(x)
print(error)

e, x, error = powerMethod(mat, minEigen=True)
print(e)
print(x)
print(error)

[[-7  2]
 [ 8 -1]]
-9.0
[ 1. -1.]
0.0
[[-2.5 -2.5]
 [ 3.5  3.5]]
1.0000000000000004
[-0.71428571  1.        ]
4.440892098500624e-16
1.0
[0.25 1.  ]
0.0


In [544]:
mat = np.array([[1, 3],
                [2, 2]])

e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat)
print(deflated)
print(e)
print(x)
print(error)

4.0
[1. 1.]
0.0
[[-1.  1.]
 [ 0.  0.]]
-1.0
[ 1. -0.]
0.0


In [545]:
mat = np.array([[3, 2, 0],
                [2, 4, 2],
                [0, 2, 5]])
e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat)
print(deflated)
print(e)
print(x)
print(error)

7.0
[0.5 1.  1. ]
0.0
[[ 2.22222222  0.44444444 -1.55555556]
 [ 0.44444444  0.88888889 -1.11111111]
 [-1.55555556 -1.11111111  1.88888889]]
4.0
[ 1.   0.5 -1. ]
0.0


In [546]:
mat = np.array([[3, 0],
                [0, 2]])

e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat)
print(deflated)
print(e)
print(x)
print(error)

3.0
[1. 0.]
0.0
[[ 0. -0.]
 [-0.  2.]]
2.0
[-0.  1.]
0.0


In [547]:
mat = np.array([[-4, 14, 0],
                [-5, 13, 0],
                [-1, 0, 2]])

e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat)
print(deflated)
print(e)
print(x)
print(error)

5.999999999999995
[ 1.          0.71428571 -0.25      ]
0.0
[[-7.81508516 11.27493917  0.95377129]
 [-7.72506083 11.05352798  0.68126521]
 [-0.04622871  0.68126521  1.76155718]]
3.0000000000000004
[1.         0.91958042 0.46853147]
0.0


In [548]:
mat = np.array([[1, 1],
                [0, -1]])

e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat)
print(deflated)
print(e)
print(x)
print(error)

0.5917690010920078
[1.         0.68984857]
1.8555881880138263
[[ 0.6686819   0.73014216]
 [-0.26985784 -1.2197986 ]]
-1.1089580496406666
[-0.41073681  1.        ]
0.0


In [549]:
mat = np.array([[87, 270, -12, -49, -276, 40],
                [-14, -45, 6, 10, 46, -4],
                [-50, -156, 4, 25, 162, -25],
                [94, 294, -5, -47, -306, 49],
                [1, 1, 3, 1, 0, 2],
                [16, 48, 1, -6, -48, 8]])

e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat)
print(deflated)
print(e)
print(x)
print(error)

4.000000000000014
[ 0.33333333  0.         -0.66666667  1.          0.          0.33333333]
2.8421709430403906e-14
[[  86.73333333  270.          -11.46666667  -49.8        -276.
    39.73333333]
 [ -14.          -45.            6.           10.           46.
    -4.        ]
 [ -49.46666667 -156.            2.93333333   26.6         162.
   -24.46666667]
 [  93.2         294.           -3.4         -49.4        -306.
    48.2       ]
 [   1.            1.            3.            1.           -0.
     2.        ]
 [  15.73333333   48.            1.53333333   -6.8         -48.
     7.73333333]]
2.999999999999783
[-0.25  1.    0.25 -0.    1.    0.75]
1.0643338062740605e-13


In [550]:
mat = np.array([[-7, 0, 0, 0],
                [0, -1, 0, 0],
                [0, 0, -25, 0],
                [0, 0, 0, -12]])

e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat)
print(deflated)
print(e)
print(x)
print(error)

deflated, e, x, error = deflate(mat, minEigen=True)
print(deflated)
print(e)
print(x)
print(error)

e, x, error = powerMethod(mat, minEigen=True)
print(e)
print(x)
print(error)

-25.0
[0. 0. 1. 0.]
0.0
[[ -7.   0.   0.   0.]
 [  0.  -1.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0. -12.]]
-12.0
[ 0.  0. -0.  1.]
0.0
[[-0.14285714  0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.         -0.04        0.        ]
 [ 0.          0.          0.         -0.08333333]]
-7.0
[ 1. -0.  0.  0.]
0.0
-1.0
[0. 1. 0. 0.]
0.0


In [551]:
mat = np.array([[1, 1, -1],
                [1, 2, 1],
                [2, -1, 1]])
e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

2.6398020042326564
[0.56298354 1.         0.07681847]
6.729129065558861e-16


In [552]:
mat = np.array([[-2, 1, 1],
                [3, -2, 0],
                [1, 3, 1]])
e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

-3.0
[-0.33333333  1.         -0.66666667]
1.4802973661668753e-16


In [553]:
mat = np.array([[3, -5],
                [-2, 4]])
e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

6.701562118716424
[ 1.         -0.74031242]
0.0


In [554]:
mat = np.array([[1, 2],
                [3, 4]])
e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

5.372281323269014
[0.45742711 1.        ]
0.0


In [555]:
mat = np.array([[2, 3],
                [5, 4]])
e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

7.0
[0.6 1. ]
0.0


In [556]:
mat = np.array([[4, 2],
                [1, 3]])
e, x, error = powerMethod(mat)
print(e)
print(x)
print(error)

5.0
[1.  0.5]
0.0
